In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
#os.symlink('/content/mnt/My Drive/colabpackages', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)
#https://colab.research.google.com/drive/1KpMDi9CjImudrzXsyTDAuRjtbahzIVjq#scrollTo=trAxoCkMLgF1

Mounted at /content/mnt


In [2]:
!pip install --target=$nb_path face-recognition
#!pip install face-recognition

     |████████████████████████████████| 100.2MB 44kB/s 
     |████████████████████████████████| 2.2MB 51.0MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
     |████████████████████████████████| 14.8MB 220kB/s 
     |████████████████████████████████| 3.6MB 45.2MB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=156751ebe6778371254050e929f46bbbd4959bf4f5921e70f6e274628e4f4dd6
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
ERROR: Operation cancelled by user


In [3]:
import face_recognition
import cv2
import numpy as np

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

import IPython
from google.colab import output
from google.colab.patches import cv2_imshow

import time
import sys

from PIL import Image
from io import BytesIO
import base64
import logging
import numpy as np
from PIL import Image as im 

ModuleNotFoundError: ignored

In [ ]:
def data_uri_to_img(uri):
  """convert base64image to numpy array"""
  try:
    image = base64.b64decode(uri.split(',')[1], validate=True)
    # make the binary image, a PIL image
    image = Image.open(BytesIO(image))
    # convert to numpy array
    image = np.array(image, dtype=np.uint8); 
    return image
  except Exception as e:
    logging.exception(e);print('\n')
    return None

In [ ]:
# Load a sample picture and learn how to recognize it.
prasad_image = face_recognition.load_image_file("/content/mnt/My Drive/colabpackages/prasad.jpg")
prasad_face_encoding = face_recognition.face_encodings(prasad_image)[0]

# Load a second sample picture and learn how to recognize it.
shanaya_image = face_recognition.load_image_file("/content/mnt/My Drive/colabpackages/shanaya1.jpg")
shanaya_face_encoding = face_recognition.face_encodings(shanaya_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
   prasad_face_encoding,
   shanaya_face_encoding
]
known_face_names = [
    "RP",
    "Shanaya"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

In [ ]:
'''
## Camera Capture
Using a webcam to capture images for processing on the runtime.
Source: https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi
'''
#process_this_frame = True

def stream_camera(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function streamCamera(quality) {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      // show the video in the HTML element
      document.body.appendChild(div);
      //Shows the video
      //div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // prints the logs to cell
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc}... `));
      }

      // Wait for Capture to be clicked.
      // await new Promise((resolve) => capture.onclick = resolve);

      for (let i = 0; i < 3; i++) {
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        img = canvas.toDataURL('image/jpeg', quality);

        // show each captured image
        //let imgTag = document.createElement('img');
        //imgTag.src = img;
        //div.appendChild(imgTag);

        //jsLog(i + "sending")
        // Call a python function and send this image
        google.colab.kernel.invokeFunction('notebook.face_algo', [img], {});
        //jsLog(i + "SENT")

        // wait for X miliseconds second, before next capture
        await new Promise(resolve => setTimeout(resolve, 250));
      }
      
      stream.getVideoTracks()[0].stop(); // stop video stream
    }
    ''')
  display(js) # make the provided HTML, part of the cell
  data = eval_js('streamCamera({})'.format(quality)) # call the streamCamera() JavaScript function

In [ ]:
def face_algo(imgB64):
  frame = data_uri_to_img(imgB64) 
  
  if frame is None:
    print("At face_algo(): image is None.")
    return
  try:
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]
    # Only process every other frame of video to save time
    #if process_this_frame:
    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    face_names = []

    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # # If a match was found in known_face_encodings, just use the first one.
        # if True in matches:
        #     first_match_index = matches.index(True)
        #     name = known_face_names[first_match_index]

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

    #process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Blue color in BGR 
        color = (255, 0, 0) 
        # Line thickness of 2 px 
        thickness = 2

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0,128,0), thickness)
     
        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image  
    cv2_imshow(frame)

  except Exception as e:
    logging.exception(e)
    print('\n')
  
# register this function, so JS code could call this
output.register_callback('notebook.face_algo', face_algo)
# put the JS code in cell and run it
stream_camera()

In [ ]:
cv2.destroyAllWindows()